<a href="https://colab.research.google.com/github/avish006/Rag-Project/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install fitz

In [ ]:
!pip install --upgrade --force-reinstall pymupdf

  Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.4 kB)
Using cached pymupdf-1.25.3-cp39-abi3-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (20.0 MB)
  Attempting uninstall: pymupdf
    Found existing installation: PyMuPDF 1.25.3
    Uninstalling PyMuPDF-1.25.3:
      Successfully uninstalled PyMuPDF-1.25.3


In [ ]:
import fitz

#Extracting text from a pdf document
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text
text = extract_text_from_pdf('/content/Attention is all you need.pdf')

In [ ]:
#Using Recursive Character Text Splitter for dividing text into chunks of text
from langchain.text_splitter import RecursiveCharacterTextSplitter
def recursive_chunking(text, chunk_size=200, overlap=100):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=overlap)
    return text_splitter.split_text(text)


In [ ]:
#Splitting the documents into chunks of text
chunks = recursive_chunking(text)

In [ ]:
#Sentence Transformer for creating vector embeddings of chunks
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('BAAI/bge-large-en-v1.5')
embeddings = [model.encode(chunk,normalize_embeddings=True).tolist() for chunk in chunks]

In [ ]:
pip install faiss-cpu

In [ ]:
import faiss
import numpy as np

def store_embeddings_faiss(embeddings):
    embedding_dim = len(embeddings[0])  # Get the size of each embedding
    index = faiss.IndexFlatL2(embedding_dim)  # Create FAISS index (L2 norm)

    np_embeddings = np.array(embeddings).astype('float32')  # Convert list to NumPy array
    index.add(np_embeddings)  # Add embeddings to FAISS

    faiss.write_index(index, "vector_store.index")  # Save index to disk
    print("Embeddings stored successfully in FAISS!")

    return index

In [ ]:
def load_faiss_index():
    return faiss.read_index("vector_store.index")

In [ ]:
index = store_embeddings_faiss(embeddings)  # Store embeddings

Embeddings stored successfully in FAISS!


In [ ]:
loaded_index = load_faiss_index()  # Load stored embeddings

In [ ]:
from openai import OpenAI
import numpy as np

# Initialize OpenRouter client
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="####",
)

In [ ]:
# Function to retrieve relevant chunks (FAISS example)
def search_faiss(index, query_embedding, top_k=6):
    query_embedding = np.array([query_embedding]).astype('float32')
    distances, indices = index.search(query_embedding, top_k)
    return indices[0]  # Top matching chunks

In [ ]:
# Function to send query to DeepSeek R1 via OpenRouter
def query_rag(user_query, index, chunks):
    query_embedding = model.encode(user_query,normalize_embeddings=True)  # Convert query to embedding
    retrieved_indices = search_faiss(index, query_embedding)  # Get top matching chunks

    # Combine retrieved text for context
    retrieved_text = " ".join([chunks[i] for i in retrieved_indices])

    # Construct full prompt with retrieved context
    full_prompt = f"""You are an advanced AI assistant with deep reasoning abilities. Use the provided context to answer the question thoughtfully. If needed, go beyond the exact wording and apply logical reasoning to infer deeper insights. Context: {retrieved_text} Question: {user_query} Your answer should be: - **Comprehensive** (Cover key details) - **Logical** (Explain reasoning clearly) - **Abstract when necessary** (Don't just summarize, but also interpret meaning)"""

    # Send request to DeepSeek R1 via OpenRouter
    completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "<YOUR_SITE_URL>",
            "X-Title": "<YOUR_SITE_NAME>",
        },
        model="deepseek/deepseek-r1:free",
        messages=[{"role": "user", "content": full_prompt}],
        temperature=0.7,  # Increase randomness for more detailed responses
        top_p=0.7 , # Allow for more diverse word choices
        presence_penalty=0.35, #Allow for newer words to be used
        frequency_penalty = 0.2, #Penalizes repeatition of words
        max_tokens= 1600
    )

    return completion.choices[0].message.content

In [ ]:
response = query_rag("List down the names of all the authors", index, chunks)
print(response)

- **Comprehensive List of Authors**:  
  The context includes authors from research papers and affiliations. By carefully parsing the text and resolving duplicates/affiliations, the unique authors are:  
  **Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Oﬁr Press, Lior Wolf, Jianpeng Cheng, Li Dong, Mirella Lapata, Romain Paulus, Caiming Xiong, Richard Socher, Nal Kalchbrenner, Lasse Espeholt, Karen Simonyan, Aaron van den Oord, Alex Graves, Kory Kavukcuoglu**.

- **Logical Reasoning**:  
  1. **Direct Extraction**: Names were extracted from email headers, affiliations, and citations.  
  2. **Deduplication**: Identified overlaps (e.g., "Noam" and "Noam Shazeer" refer to the same person via matching emails).  
  3. **Context Clustering**: Authors like Niki Parmar and Jakob Uszkoreit appear in both the Google Research header and the *Attention Is All You Need* section, confirming their uniqueness.  

- **Abstract Interpretation**:  
  The list 

In [ ]:
#Hybrid Searching for improved response quality ,
#Enable Multi-Step Reasoning with Recursive Calls, If question is complex, generate reasoning steps first, then refine its answer (Method: Multi-Step RAG (RAG + Self-Refinement)
#Chain Of Thought